## Syntaxe des environnements

#### Motivation

- Jusqu'à présent, nous avons utilisé des environnements prédéfinis comme Frozen Like et Google RecSim.
- Pour utiliser RL sur notre propre problème, nous ne pouvons utiliser aucun de ces environnements.
- Nous devrons définir notre propre environnement avec Python.

#### Revue de Frozen Lake

- Rappelle-toi l'environnement de Frozen Lake, du module 1 :

In [1]:
import gym
env = gym.make("FrozenLake-v1")

#### Revue de Frozen Lake

- OpenAI Gym est une source ouverte, nous avons donc pu consulter le [code source de Frozen Lake] (https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py).
- Cependant, il est compliqué et contient bien plus que ce dont nous avons besoin.
- Créons notre propre environnement appelé Frozen Pond avec les composants de base de Frozen Lake.

#### Composants d'une Env

Décisions conceptuelles :

- Espace d'observation
- Espace d'action

En Python, nous devrons implémenter, au moins :

- constructeur
- `reset()`
- `step()`

En pratique, nous pouvons aussi vouloir d'autres méthodes, comme `render()`

#### Décisions conceptuelles

Dans ce cas, comme nous imitons le Lac gelé, l'espace d'observation et l'espace d'action sont déjà décidés.

In [2]:
observation_space = gym.spaces.Discrete(16)
action_space = gym.spaces.Discrete(4)

Plus tard dans ce cours, nous nous plongerons plus profondément dans ces décisions !

#### Coding it up

In [3]:
import gym

class FrozenPond(gym.Env):
    pass

- Remarque que nous commençons par sous-classer `gym.Env`.
- Facultatif : Tu peux lire sur les objets, l'héritage et les sous-classes.
- L'essentiel : Il s'agit d'un `gym.Env` de base et nous pouvons en écraser les caractéristiques.

#### Constructeur

- Le constructeur est appelé lorsque nous créons un nouvel objet `FrozenPond`.
- C'est ici que nous définissons l'espace d'observation et l'espace d'action.

In [4]:
import gym

class FrozenPond(gym.Env):
    def __init__(self, env_config=None):
        self.observation_space = gym.spaces.Discrete(16)
        self.action_space = gym.spaces.Discrete(4)        

- Pour la compatibilité avec RLlib, le constructeur doit prendre un `env_config` 
- Nous allons simplement ignorer cet argument pour l'instant.

_COPY #### Reset

- La prochaine méthode dont nous aurons besoin est la réinitialisation.
- Le constructeur définit des paramètres permanents comme l'espace d'observation.
- `reset` configure chaque nouvel épisode.
- Il y a une certaine liberté entre les deux, par exemple pour définir l'emplacement du but.
- Si quelque chose _pourrait_ changer, nous le mettrons dans `reset`.

In [5]:
# HIDDEN
import numpy as np

In [6]:
class FrozenPond(gym.Env):
    def reset(self):
        self.player = (0, 0) # the player starts at the top-left
        self.goal = (3, 3)   # goal is at the bottom-right
        
        self.holes = np.array([
            [0,0,0,0], # FFFF 
            [0,1,0,1], # FHFH
            [0,0,0,1], # FFFH
            [1,0,0,0]  # HFFF
        ])
        
        return 0 # to be changed to return self.observation()

_COPY #### Reset

Testons cela :

In [7]:
fp = FrozenPond()

In [8]:
fp.reset()

0

In [9]:
fp.holes

array([[0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])

Ça a l'air bien.

#### Étape

- La dernière méthode dont nous avons besoin est `step`.
- C'est la méthode la plus compliquée qui contient la logique de base.
- Rappelle-toi que `step` renvoie 4 choses :
  1. Observation
  2. Récompense
  3. Drapeau fait
  4. Info supplémentaire (nous l'ignorerons)
- Pour plus de clarté, nous allons écrire des méthodes d'aide pour observation, récompense et fait, plus une méthode d'aide supplémentaire 

#### Étape : observation

Rappelle-toi que l'observation est un indice de 0 à 15 :

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

Nous pouvons coder cela comme suit :

In [10]:
class FrozenPond(gym.Env):
    def observation(self):
        return 4*self.player[0] + self.player[1]

Par exemple, si le joueur se trouve à (2,1), nous renvoyons le message suivant

In [11]:
4*2 + 1

9

Remarque : maintenant que `self.observation` est implémenté, nous devrions changer `reset` en `return self.observation()` plutôt que `return 0` pour un code de meilleure qualité.

#### Étape : récompense

En suivant l'exemple de Frozen Lake, la récompense sera de 1 si l'agent atteint l'objectif, et de 0 sinon :

In [12]:
class FrozenPond(gym.Env):
    def reward(self):
        return int(self.player == self.goal)

Nous modifierons cette fonction de récompense plus tard dans le module !

####. Étape : faite

- Nous avons également besoin de savoir quand un épisode est terminé 
- Selon Frozen Lake, l'épisode est terminé lorsque l'agent atteint l'objectif ou tombe dans l'étang.

In [13]:
class FrozenPond(gym.Env):
    def done(self):
        return self.player == self.goal or self.holes[self.player] == 1

#### Étape : emplacements valides

Enfin, pour simplifier la méthode `step`, nous allons écrire une méthode auxiliaire appelée `is_valid_loc` qui vérifie si un emplacement particulier est dans les limites (de 0 à 3 dans chaque dimension).

In [14]:
class FrozenPond(gym.Env):
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3:
            return True
        else:
            return False

#### Étape : assembler le tout

- En utilisant les éléments ci-dessus, nous pouvons maintenant écrire la méthode `step`.
- `step` prend une _action_, met à jour l'_état_ et renvoie l'observation, la récompense, le drapeau "done" et des informations supplémentaires (ignorées).
- Rappelle-toi comment les actions sont codées : 0 pour gauche, 1 pour bas, 2 pour droite, 3 pour haut.
- Nous allons implémenter un étang gelé **non glissant** ; en d'autres termes, déterministe plutôt que stochastique.

In [15]:
class FrozenPond(gym.Env):
    def step(self, action):
        # Compute the new player location
        if action == 0:   # left
            new_loc = (self.player[0], self.player[1]-1)
        elif action == 1: # down
            new_loc = (self.player[0]+1, self.player[1])
        elif action == 2: # right
            new_loc = (self.player[0], self.player[1]+1)
        elif action == 3: # up
            new_loc = (self.player[0]-1, self.player[1])
        else:
            raise ValueError("Action must be in {0,1,2,3}")
        
        # Update the player location only if you stayed in bounds
        # (if you try to move out of bounds, the action does nothing)
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        
        # Return observation/reward/done
        return self.observation(), self.reward(), self.done(), {}

#### Succès !

- C'est fait ! Nous avons implémenté les pièces nécessaires dans Frozen Pond 
  - constructeur
  - `reset`
  - `step`
- Nous ajouterons également une fonction facultative `render` afin de pouvoir dessiner l'état :

In [16]:
class FrozenPond(gym.Env):
    def render(self):
        for i in range(4):
            for j in range(4):
                if (i,j) == self.goal:
                    print("⛳️", end="")
                elif (i,j) == self.player:
                    print("🧑", end="")
                elif self.holes[i,j]:
                    print("🕳", end="")
                else:
                    print("🧊", end="")
            print()

- Pour le plaisir, nous allons utiliser des émojis dans le rendu de notre client.
- Le joueur est 🧑, le but est ⛳️, le segment de lac gelé est 🧊, le trou est 🕳.

#### Tester notre implémentation

In [17]:
# HIDDEN
from envs_03 import FrozenPond

In [18]:
env = FrozenPond()
env.reset()
env.render()

🧑🧊🧊🧊
🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🧊🧊⛳️


#### Tester notre implémentation

Testons la méthode `step` :

In [19]:
env.step(2) # 0=left / 1=down / 2=right / 3=up

(1, 0, False, {'player': (0, 1), 'goal': (3, 3)})

In [20]:
env.render()

🧊🧑🧊🧊
🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🧊🧊⛳️


Ça a l'air bien !

#### Tester notre implémentation

Comparons directement les deux environnements :

In [21]:
lake = gym.make("FrozenLake-v1", is_slippery=False)
pond = FrozenPond()

lake.reset()
pond.reset()

print("Iter | gym obs / our obs | gym reward / our reward | gym done / our done")
for i, a in enumerate([0, 2, 2, 1, 1, 1, 1, 2]):
    lake_obs, lake_rew, lake_done, _ = lake.step(a)
    pond_obs, pond_rew, pond_done, _ = pond.step(a)
    print("%2d   |      %2d / %2d      |          %d / %d        |      %5s / %5s" % \
          (i, lake_obs, pond_obs, lake_rew, pond_rew, lake_done, pond_done))

Iter | gym obs / our obs | gym reward / our reward | gym done / our done
 0   |       0 /  0      |          0 / 0        |      False / False
 1   |       1 /  1      |          0 / 0        |      False / False
 2   |       2 /  2      |          0 / 0        |      False / False
 3   |       6 /  6      |          0 / 0        |      False / False
 4   |      10 / 10      |          0 / 0        |      False / False
 5   |      14 / 14      |          0 / 0        |      False / False
 6   |      14 / 14      |          0 / 0        |      False / False
 7   |      15 / 15      |          1 / 1        |       True /  True


Ils sont identiques !

#### Tester notre implémentation

- RLlib est également livré avec un vérificateur d'env
- Cela ne nous dira pas si notre env est identique à celle de Frozen Lake
- Mais il effectuera plusieurs vérifications utiles :

In [22]:
from ray.rllib.utils.pre_checks.env import check_env

In [23]:
check_env(pond)

2022-08-27 16:08:55,501	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


- Toutes les vérifications ont été passées, sauf cet avertissement concernant la longueur maximale d'un épisode.
- Nous pouvons/devrions le définir pour que les épisodes ne puissent pas devenir arbitrairement longs.

#### Nombre maximum d'étapes par épisode

- Pour définir un nombre maximum de pas par épisode, nous pouvons utiliser un `gym` _wrapper_.
- Les wrappers sont des moyens pratiques de modifier les environnements, y compris les observations, les actions et les récompenses.
- Ici, nous allons utiliser le wrapper `TimeLimit` pour définir une limite de pas.

In [24]:
from gym.wrappers import TimeLimit

pond_5 = TimeLimit(pond, max_episode_steps=5)

Nous pouvons vérifier que cela sera fait après 5 étapes, même si le but n'est pas atteint :

In [25]:
pond_5.reset()
for i in range(5):
    print(pond_5.step(0))

(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, True, {'player': (0, 0), 'goal': (3, 3), 'TimeLimit.truncated': True})


#### Nombre maximum d'étapes par épisode

Une limite de pas plus raisonnable pourrait être de 50, plutôt que de 5.

In [26]:
pond_50 = TimeLimit(pond, max_episode_steps=50)

- Pour info : il est également possible de définir cette limite dans RLlib, juste à des fins d'entraînement.
- Cela se fait avec le paramètre `"horizon"` dans la configuration de l'entraîneur.

#### Appliquons ce que nous avons appris !

## Les récompenses de Frozen Pond
<!-- multiple choice -->

Dans le lac (et l'étang) gelé, la récompense est de 1 lorsque l'agent atteint l'objectif, et de 0 sinon. L'agent doit apprendre à éviter les trous, mais il n'y a en fait aucune récompense négative à tomber dans un trou - c'est la même récompense nulle que de marcher dans un morceau sûr du lac gelé ! Pourquoi cette configuration fonctionne-t-elle toujours, même si la récompense est la même pour marcher dans un trou ou sur la terre ferme ?

- [ ] Une fois que l'agent est tombé dans un trou, il est coincé. Il peut faire d'autres actions, mais elles ne font rien. L'agent apprend donc à éviter les trous.
- [ ] Une récompense de 0 est la plus petite récompense possible ; par conséquent, lorsque l'agent reçoit une récompense de 0 en tombant dans un trou, il sait immédiatement que tomber dans un trou est une mauvaise chose.
- [La pénalité liée au fait de tomber dans un trou est indirecte, car l'épisode se termine par une récompense de zéro, ce qui signifie qu'il renonce à la récompense potentielle de 1 qu'il pourrait obtenir en atteignant son objectif. L'agent apprend qu'en tombant dans un trou, il perd des récompenses _futures_.
- [Les agents RL préfèrent les épisodes plus longs. Lorsque l'agent tombe dans le trou, l'épisode se termine immédiatement, ce que l'agent apprend à éviter.

## Étang contre labyrinthe
<!-- coding exercise -->

Disons que nous voulons transformer notre environnement d'étang en un environnement de labyrinthe. Dans ce cas, nous avons des murs au lieu de trous. La seule différence entre l'étang et le labyrinthe est le comportement des trous par rapport aux murs. Dans l'étang gelé, marcher dans un trou met fin à l'épisode. Dans l'environnement du labyrinthe, marcher dans un mur ne fait rien (c'est-à-dire que l'action ne change pas l'emplacement de l'agent, tout comme essayer de marcher sur le bord de la carte). Pour transformer notre lac gelé en labyrinthe, nous devrons modifier deux méthodes : `done` et `is_valid_loc`.

Tu trouveras ci-dessous les méthodes `done` et `step` que nous avons vues dans les diapositives ci-dessus. Modifie-les pour que nous ayons maintenant un labyrinthe avec le comportement décrit ci-dessus : marcher dans un mur ne fait rien.

Note que la classe `Maze` hérite de toutes les autres méthodes de `FrozenPond`, tu peux donc la tester !

In [27]:
# EXERCISE
from envs_03 import FrozenPond


class Maze(FrozenPond):
    def done(self):
        return self.player == self.goal or self.holes[self.player] == 1
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3:
            return True
        else:
            return False
    
pond = FrozenPond()
pond.reset()
pond.step(1)
print(pond.step(2))

maze = Maze()
maze.reset()
maze.step(1)
print(maze.step(2))

(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})
(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})


In [28]:
# SOLUTION
from envs_03 import FrozenPond


class Maze(FrozenPond):   
    def done(self):
        return self.player == self.goal
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3 and not self.holes[location]:
            return True
        else:
            return False
    
pond = FrozenPond()
pond.reset()
pond.step(1)
print(pond.step(2))

maze = Maze()
maze.reset()
maze.step(1)
print(maze.step(2))

(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})
(4, 0, False, {'player': (1, 0), 'goal': (3, 3)})
